In [1]:
import json
import pandas as pd
import numpy as np
import openai
import pickle 
import os
import time
from tqdm import tqdm

In [ ]:
#!pip install openai

## Get SQuAD Dataset

In [2]:
#get SQuAD v2
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2023-02-03 16:22:56--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   201MB/s    in 0.2s    

2023-02-03 16:22:58 (201 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2023-02-03 16:22:59--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [3]:
num_questions = 0
num_posible = 0
num_imposible = 0

unique_qa = []
unique_contexts = []

In [4]:
'''
NOTE: 
This is some generic code to retieve contexts questions and answers from the dataset.
More information that needed but keeping it here in case someone wants to expand the code for other purposes.
'''

def get_data(path):  
    #read each file and retrieve the contexts, qustions and answers
  with open(path, 'rb') as f:
    raw_data = json.load(f)

  contexts = []
  answerable_questions = []
  answers = []
  unanswerable_questions = []
  unanswerable_contexts = []


  num_q = 0
  num_pos = 0
  num_imp = 0

  for group in raw_data['data']:
    for paragraph in group['paragraphs']:
      context = paragraph['context']
      context = context.replace('\n','')
      if context not in unique_contexts:
            unique_contexts.append(context)
      for qa in paragraph['qas']:
        question = qa['question']
        num_q  = num_q  +1
        if qa['is_impossible'] == True:
            num_imp = num_imp +1
            unanswerable_questions.append(question)
            unanswerable_contexts.append(context)
        else:
            for answer in qa['answers']:
                tup = (question, answer)
                if tup not in unique_qa:
                    num_pos = num_pos +1
                    unique_qa.append(tup)
                    contexts.append(context)
                    answerable_questions.append(question)
                    answers.append(answer)

  return num_q, num_pos, num_imp, contexts, answerable_questions, answers, unanswerable_questions,unanswerable_contexts

In [5]:
num_q, num_pos, num_imp, valid_contexts, valid_questions, valid_answers, invalid_questions, invalid_contexts = get_data('train-v2.0.json')
num_questions  = num_questions + num_q
num_posible = num_posible + num_pos
num_imposible = num_imposible  + num_imp

print(f"Total number of questions: {num_questions}")
print(f"Total number of Answerable questions: {num_posible}")
print(f"Total number of impossible questions: {num_imposible}")

Total number of questions: 130319
Total number of Answerable questions: 86797
Total number of impossible questions: 43498


In [6]:
print(f"Total number of unique contexts: {len(unique_contexts)}")

Total number of unique contexts: 19029


In [7]:
test_rec = 5
print(f"Context: {valid_contexts[test_rec]}")
print(f"Question: {valid_questions[test_rec]}")
print(f"Answer: {valid_answers[test_rec]}")

Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Question: In what R&B group was she the lead singer?
Answer: {'text': "Destiny's Child", 'answer_start': 320}


In [8]:
test_rec = 5
print(f"Context: {invalid_contexts[test_rec]}")
print(f"Question: {invalid_questions[test_rec]}")


Context: The story focuses on series protagonist Link, who tries to prevent Hyrule from being engulfed by a corrupted parallel dimension known as the Twilight Realm. To do so, he takes the form of both a Hylian and a wolf, and is assisted by a mysterious creature named Midna. The game takes place hundreds of years after Ocarina of Time and Majora's Mask, in an alternate timeline from The Wind Waker.
Question: What land does Ocarina serve to protect?


## Connect to Open AI

In [9]:
COMPLETIONS_MODEL = 'text-davinci-003' #"text-curie-001"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [10]:
KEY = '<GET THE KEY FROM THE OPEN AI ACCOUNT>'

In [11]:
os.environ["OPENAI_API_KEY"] = KEY

In [12]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Test 

In [14]:
context = """On December 13, 2013, Beyoncé unexpectedly released her eponymous fifth studio album on the iTunes Store without any prior announcement or promotion. The album debuted atop the Billboard 200 chart, giving Beyoncé her fifth consecutive number-one album in the US. This made her the first woman in the chart\'s history to have her first five studio albums debut at number one. Beyoncé received critical acclaim and commercial success, selling one million digital copies worldwide in six days; The New York Times noted the album\'s unconventional, unexpected release as significant. Musically an electro-R&B album, it concerns darker themes previously unexplored in her work, such as "bulimia, postnatal depression [and] the fears and insecurities of marriage and motherhood". The single "Drunk in Love", featuring Jay Z, peaked at number two on the Billboard Hot 100 chart. In April 2014, after much speculation in the weeks before, Beyoncé and Jay Z officially announced their On the Run Tour. It served as the couple\'s first co-headlining stadium tour together. On August 24, 2014, she received the Video Vanguard Award at the 2014 MTV Video Music Awards. Knowles also took home three competitive awards: Best Video with a Social Message and Best Cinematography for "Pretty Hurts", as well as best collaboration for "Drunk in Love". In November, Forbes reported that Beyoncé was the top-earning woman in music for the second year in a row—earning $115 million in the year, more than double her earnings in 2013. Beyoncé was reissued with new material in three forms: as an extended play, a box set, as well as a full platinum edition.
In September 2010, Beyoncé made her runway modelling debut at Tom Ford\'s Spring/Summer 2011 fashion show. She was named "World\'s Most Beautiful Woman" by People and the "Hottest Female Singer of All Time" by Complex in 2012. In January 2013, GQ placed her on its cover, featuring her atop its "100 Sexiest Women of the 21st Century" list. VH1 listed her at number 1 on its 100 Sexiest Artists list. Several wax figures of Beyoncé are found at Madame Tussauds Wax Museums in major cities around the world, including New York, Washington, D.C., Amsterdam, Bangkok, Hollywood and Sydney.
Beyoncé is believed to have first started a relationship with Jay Z after a collaboration on "\'03 Bonnie & Clyde", which appeared on his seventh album The Blueprint 2: The Gift & The Curse (2002). Beyoncé appeared as Jay Z\'s girlfriend in the music video for the song, which would further fuel speculation of their relationship. On April 4, 2008, Beyoncé and Jay Z were married without publicity. As of April 2014, the couple have sold a combined 300 million records together. The couple are known for their private relationship, although they have appeared to become more relaxed in recent years. Beyoncé suffered a miscarriage in 2010 or 2011, describing it as "the saddest thing" she had ever endured. She returned to the studio and wrote music in order to cope with the loss. In April 2011, Beyoncé and Jay Z traveled to Paris in order to shoot the album cover for her 4, and unexpectedly became pregnant in Paris.
"""

question = """When did Beyonce start becoming popular?"""

In [16]:
prompt = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don\'t know."

Context:
{context}

Q: {question}
A:""".format(context=context,question=question)

prompt

'Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don\'t know."\n\nContext:\nOn December 13, 2013, Beyoncé unexpectedly released her eponymous fifth studio album on the iTunes Store without any prior announcement or promotion. The album debuted atop the Billboard 200 chart, giving Beyoncé her fifth consecutive number-one album in the US. This made her the first woman in the chart\'s history to have her first five studio albums debut at number one. Beyoncé received critical acclaim and commercial success, selling one million digital copies worldwide in six days; The New York Times noted the album\'s unconventional, unexpected release as significant. Musically an electro-R&B album, it concerns darker themes previously unexplored in her work, such as "bulimia, postnatal depression [and] the fears and insecurities of marriage and motherhood". The single "Drunk in Love", featuring Jay Z, peaked at numb

In [18]:
# Fins andwer to question
ret = openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

ret

'Beyonce started becoming popular in the early 2000s, after the release of her first solo album, Dangerously in Love, in 2003.'

## Create Embeddings

In [19]:
def get_embedding(text):
    result = openai.Embedding.create(
      model=EMBEDDING_MODEL,
      input=text
    )
    return result["data"][0]["embedding"]

In [ ]:
#create embeddings for all the contexts.
tot_contexts = len(unique_contexts)
doc_embeddings = {}
for i in tqdm(range (0,tot_contexts), total=tot_contexts, desc="status:"):
    doc_embeddings[i] = get_embedding(unique_contexts[i])

In [30]:
#save embeddings so you don't have to run more than once
with open('saved_embeddings.pkl', 'wb') as f:
    pickle.dump(doc_embeddings, f)

In [28]:
#convert the contexts to a dictionary and save them as well
all_contexts = {}
i = 0
for c in unique_contexts:
    all_contexts[i] = c
    i = i + 1

#save contexts
with open('saved_contexts.pkl', 'wb') as f:
    pickle.dump(all_contexts, f)

## LOAD SAVED EMBEDDINGS

In [31]:
with open('saved_embeddings.pkl', 'rb') as f:
    loaded_embeddings = pickle.load(f)

In [32]:
with open('saved_contexts.pkl', 'rb') as f:
    loaded_contexts = pickle.load(f)

In [35]:
print(f"Length of each embedding vector: {len(doc_embeddings[29])}")

Length of each embedding vector: 1536


## Find most similar docs

In [36]:
def vector_similarity(x, y) -> float:
    """
    Returns the similarity between two vectors.
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

In [38]:
def order_document_sections_by_query_similarity(query):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), loaded_contexts[doc_index]) for doc_index, doc_embedding in loaded_embeddings.items()
    ], reverse=True)
    
    return document_similarities

## Construct Prompt

In [41]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "

separator_len = len(SEPARATOR)

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [42]:
def construct_prompt(question) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, document_section in most_relevant_document_sections[:3]:
        # Add top 3 contexts        
        chosen_sections.append(SEPARATOR + document_section)
            
    context = "".join(chosen_sections)
    
    out_prompt = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."
    
    Context:
    {context}
    
    QUESTION: {question}
    ANSWER:
    """.format(context=context,question=question)
    
    return out_prompt

## FINAL ANSWER RETRIEVAL FUNCTION

In [50]:
def get_answer(question):
    prompt = construct_prompt(question)
    
    ret = openai.Completion.create(
        prompt=prompt,
        temperature=0,
        max_tokens=500,
        model= COMPLETIONS_MODEL
    )["choices"][0]["text"].strip(" \n")
    
    return ret
    

In [51]:
get_answer("When did Beyonce start becoming popular?")
#'Beyoncé first started becoming popular in the early 2000s, after the release of her debut solo album Dangerously in Love in 2003.'

'Beyoncé first started becoming popular in the early 2000s, after the release of her debut solo album Dangerously in Love in 2003.'

In [52]:
get_answer("Whe was the lead singer of destiny's child?")
#'Beyoncé Giselle Knowles-Carter'

'Beyoncé Giselle Knowles-Carter'

In [53]:
get_answer("Which artist did Beyonce marry?")
#'Beyoncé married Jay Z on April 4, 2008.'

'Beyoncé married Jay Z on April 4, 2008.'

In [54]:
get_answer("Whe killed Roger Rabbit?")
#"I don't know."

"I don't know."

In [55]:
get_answer("When did the Norse land in England?")
#"I don't know."

"I don't know."